## UNWARPING

In [1]:
import numpy as np
import cv2
import glob
from tqdm import tqdm
import pandas as pd

def unwarp(src, show=False):
    chessboard_size = (7, 5)
    # termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((chessboard_size[0]*chessboard_size[1],3), np.float32)
    objp[:,:2] = np.mgrid[0:chessboard_size[0],0:chessboard_size[1]].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    # cap = cv2.VideoCapture('chessboard.mp4')
    # nb_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # nb_frames = 10
    # img = None
    # for i in tqdm(range(nb_frames)):
    #     ret,frame = cap.read()
    #     if i == 0:
    #         img = frame

    #     gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #     # Find the chess board corners
    #     ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    #     # If found, add object points, image points (after refining them)
    #     if ret == True:
    #         #print('chessboard found!')
    #         objpoints.append(objp)
    #         corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
    #         imgpoints.append(corners2)
    #         #img = cv2.drawChessboardCorners(img, chessboard_size, corners2,ret)
    # #else:
    # #   print('chessboard not found...')
    # cap.release()
    # print('cap.release()')   
    images = glob.glob('calibration/small_chessboard_images/*.jpg') #glob.glob('*.jpg')
    #print(images)
    for fname in images:
        img = cv2.imread(fname)
        dims = img.shape
        height = dims[0]
        width = dims[1]
        channels = dims[2]
        # cv2.namedWindow('image', cv2.WINDOW_NORMAL)
        # cv2.resizeWindow('image', width,height)
        # cv2.imshow('image',img)
        # cv2.waitKey(0)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert to a gray scale image
        # cv2.namedWindow('gray', cv2.WINDOW_NORMAL)
        # cv2.resizeWindow('gray', width,height)
        # cv2.imshow('gray',gray)
        # cv2.waitKey(0)
        
        #hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  
        #black = cv2.inRange(hsv, np.array([0, 0, 0]),  np.array([180, 255, 30]))
        # cv2.namedWindow('black', cv2.WINDOW_NORMAL)
        # cv2.resizeWindow('black', width,height)
        # cv2.imshow('black',black)
        # cv2.waitKey(0)
    #     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        #Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            print('chessboard found!')
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            imgpoints.append(corners2)
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, chessboard_size, corners2,ret)
            # cv2.namedWindow('chessboard', cv2.WINDOW_NORMAL)
            # cv2.resizeWindow('chessboard', width,height)
            # cv2.imshow('chessboard',img)
            # cv2.waitKey(0)
        else:
            print('no chessboard found...')

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
    #img = cv2.imread('small-test.png')
    h,  w = src.shape[:2]
    newcameramtx, roi= cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    # undistort
    dst = cv2.undistort(src, mtx, dist, None, newcameramtx)
    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    if show:
        cv2.imshow('src',src)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imshow('dst',dst)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        #cv2.imwrite('calibresult.png',dst)
        # mean_error = 0
        # for i in range(len(objpoints)):
        #     imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        #     error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        #     mean_error += error
        # print ("total error: ", mean_error/len(objpoints))
    return dst
frame = cv2.imread('frames/25H-35L/image_group_27_frame_3636.0.jpg')
unwarped_frame = unwarp(frame, False)

chessboard found!
chessboard found!
chessboard found!
chessboard found!
chessboard found!
no chessboard found...
chessboard found!


## MARKERS

In [4]:

import time
import math

    
def getperpcoord(ax, ay, bx, by, startLength, endLength):
    vx = bx-ax
    vx_offset = vx/2.0
    vy = by-ay
    vy_offset = vy/2.0
    #print(str(vx)+" "+str(vy))
   # if(vx == 0 or vy == 0):
   #     return 0, 0, 0, 0
    mag = math.sqrt(vx*vx + vy*vy)
    vx = vx / mag
    vy = vy / mag
    temp = vx
    vx = 0-vy
    vy = temp
    cx = int(bx + (abs(bx-ax)/2.0) + vx * endLength)
    cy = int(by + vy * endLength)

    dx = int(bx + (abs(bx-ax)/2.0)  + vx * startLength) #- vx * length
    dy = int(by + vy * startLength)

    c = (cx, cy) if cy < dy else (dx, dy)
    d = (dx, dy) if dy > cy else (cx, cy)
    return c, d



def getMarkersCenters(src_image):
    dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_100)
    parameters =  cv2.aruco.DetectorParameters()
    detector = cv2.aruco.ArucoDetector(dictionary, parameters)
    markersCorners, markerIds, rejectedCandidates = detector.detectMarkers(src_image)
    markersCenters = []
    markerSideMillimeters = 47.0 #mm
    ratioPixelToMillimeters = 0
    ratioMillimetersToPixels = 0
    for markerCorners in markersCorners:
        markerAreaPixels = cv2.contourArea(markerCorners)
        markerSidePixels = math.sqrt(markerAreaPixels)
        ratioPixelToMillimeters += markerSidePixels/markerSideMillimeters
        ratioMillimetersToPixels += markerSideMillimeters/markerSidePixels
        M = cv2.moments(markerCorners)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            markersCenters.append((cx, cy))
    if len(markersCorners):
        ratioPixelToMillimeters /= len(markersCorners)
        ratioMillimetersToPixels /= len(markersCorners)
    return markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels

def loopGetMarkersCenters(src_image, nb_markers, show=False):
    #look for markers
    valueWhite = 0
    markersFound = False
    adjusted_image = None #cv2.cvtColor(dst, cv2.COLOR_BGR2HSV)
    markersCenters = []
    ratioPixelToMillimeters = 0
    ratioMillimetersToPixels = 0
    adjusted_src_image = None
    while not markersFound:
        adjusted_src_image = src_image.copy()
        lower_white = np.array([valueWhite,valueWhite,valueWhite])
        upper_white = np.array([255,255,255])
        adjusted_src_mask = cv2.inRange(adjusted_src_image, lower_white, upper_white)
        adjusted_src_image = cv2.bitwise_and(adjusted_src_image,adjusted_src_image, mask=adjusted_src_mask)
        markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels = getMarkersCenters(adjusted_src_image)
        markersFound = (len(markersCenters) == nb_markers)
        valueWhite += 1
        if valueWhite > 255:
            return [], 0, 0

    if show:
        print('nbMarkersCenters: %d' %len(markersCenters))
        print('ratioPixelToMillimeters: %f' %ratioPixelToMillimeters)
        print('ratioMillimetersToPixels: %f' %ratioMillimetersToPixels)
        cv2.namedWindow('adjusted_src_image', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('adjusted_src_image', adjusted_src_image.shape[1], adjusted_src_image.shape[0])
        cv2.imshow('adjusted_src_image',adjusted_src_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels


def getVerticalLine(src_image, markersCenters, ratioPixelToMillimeters, show=False):
    groundToWoodPlateMillimeters = 20
    woodToDisplayStartMillimeters = 130
    DisplayStartToDisplayEndMillimeters = 540 #500 true 
    groundToMarkersMillimeters = 70 #60 true
    groundToDisplayStartMillimeters = (groundToWoodPlateMillimeters + woodToDisplayStartMillimeters) - groundToMarkersMillimeters
    groundToDisplayEndMillimeters = groundToDisplayStartMillimeters + DisplayStartToDisplayEndMillimeters
    groundToDisplayStartPixels = groundToDisplayStartMillimeters * ratioPixelToMillimeters
    groundToDisplayEndPixels = groundToDisplayEndMillimeters * ratioPixelToMillimeters

    a = markersCenters[0] if markersCenters[0][0] >  markersCenters[1][0] else markersCenters[1]
    b = markersCenters[0] if markersCenters[0][0] <  markersCenters[1][0] else markersCenters[1]

    c, d = getperpcoord(
        a[0],
        a[1], 
        b[0], 
        b[1], 
        groundToDisplayStartPixels,
        groundToDisplayEndPixels,
        ) 

    if show:
        draw_src_image = src_image.copy()
        cv2.circle(draw_src_image, markersCenters[0], 7, (0, 0, 255), -1)
        cv2.circle(draw_src_image, markersCenters[1], 7, (0, 0, 255), -1)
        cv2.line(draw_src_image, markersCenters[0], markersCenters[1], (0, 0, 255),  1)
        cv2.line(draw_src_image, c, d, (255, 0, 0),  10)
        cv2.namedWindow('draw_src_image', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('draw_src_image', draw_src_image.shape[1], draw_src_image.shape[0])
        cv2.imshow('draw_src_image',draw_src_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return c, d

NB_MARKERS = 2
markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels = loopGetMarkersCenters(unwarped_frame, NB_MARKERS, True)
assert NB_MARKERS == len(markersCenters)
lowerCenter, upperCenter = getVerticalLine(unwarped_frame,  markersCenters, ratioPixelToMillimeters, show=True)
print(lowerCenter, upperCenter)


nbMarkersCenters: 2
ratioPixelToMillimeters: 0.984014
ratioMillimetersToPixels: 1.016275
(929, 221) (928, 753)


## TRANSFORM

In [12]:
MIN_HOLE_RADIUS_PIXELS = 189/2.0  * ratioPixelToMillimeters # 205.782cm
CENTER_LINE_ERROR = upperCenter[0] - lowerCenter[0]

def angles(u, v): 
  #using the arccos function from numpy
  return np.arccos(u.dot(v)/(np.linalg.norm(u)*np.linalg.norm(v)))

def rotate(src_image, lowerCenter, upperCenter, show=False):
    d_to_c_error = upperCenter[0] - lowerCenter[0]
    u = np.array([0, upperCenter[1] - lowerCenter[1]])
    v = np.array([CENTER_LINE_ERROR, upperCenter[1] - lowerCenter[1]])
    f_angle = angles(u,v)
    angle = math.degrees(f_angle)
    print("the vectors are=",u,"and",v)
    print("the angle between the two vectors is=",angle)
    if show:
        draw_src_image = src_image.copy()
        cv2.line(draw_src_image, lowerCenter, (lowerCenter[0], upperCenter[1]), (255, 0, 255),  10)
        cv2.line(draw_src_image, lowerCenter, upperCenter, (255, 0, 0),  10)
        cv2.namedWindow('draw_src_image', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('draw_src_image', draw_src_image.shape[1], draw_src_image.shape[0])
        cv2.imshow('draw_src_image',draw_src_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    # rotate our image by 45 degrees around the center of the image
    M = cv2.getRotationMatrix2D(lowerCenter, -angle, 1.0)
    rotated_src_image = src_image.copy()
    cv2.line(rotated_src_image, lowerCenter, (lowerCenter[0], upperCenter[1]), (255, 0, 255),  10)
    cv2.line(rotated_src_image, lowerCenter, upperCenter, (255, 0, 0),  10)
    rotated_src_image = cv2.warpAffine(rotated_src_image, M, (rotated_src_image.shape[1], rotated_src_image.shape[0]))
    if show:
        cv2.namedWindow('rotated_src_image', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('rotated_src_image', rotated_src_image.shape[1], rotated_src_image.shape[0])
        cv2.imshow('rotated_src_image',rotated_src_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return rotated_src_image


def crop(src_image, lowerCenter, upperCenter, direction=0, show=False):
    cropped_src_image = src_image.copy()
    if show: 
        cv2.namedWindow('cropped_src_image', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('cropped_src_image', cropped_src_image.shape[1], cropped_src_image.shape[0])
        cv2.imshow('cropped_src_image',cropped_src_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    y_offset = 0
    min_y = min(lowerCenter[1], upperCenter[1])- y_offset
    print('min_y:%d' %min_y)
    max_y = max(lowerCenter[1], upperCenter[1]) + y_offset
    print('max_y:%d' %max_y)
    min_x = min(lowerCenter[0], upperCenter[0])
    print('min_x:%d' %min_x)
    max_x = max(lowerCenter[0], upperCenter[0])
    print('max_x:%d' %max_x)
    delta_x = upperCenter[0] - lowerCenter[0]

    cropped_src_image = cropped_src_image[lowerCenter[1]:upperCenter[1], lowerCenter[0]::] if direction else cropped_src_image[lowerCenter[1]:upperCenter[1], 0:lowerCenter[0]]
    #cropped_src_image = cropped_src_image[min_y:max_y, 1200:(max(lowerCenter[0], upperCenter[0]) + CENTER_LINE_ERROR) + 1] 
    cropped_src_dims = cropped_src_image.shape
    cropped_src_height = cropped_src_dims[0]
    print('cropped_src_height:%d' %cropped_src_height)
    cropped_src_width = cropped_src_dims[1]
    print('cropped_src_width:%d' %cropped_src_width)
    if show: 
        cv2.namedWindow('cropped_src_image', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('cropped_src_image', cropped_src_image.shape[1], cropped_src_image.shape[0])
        cv2.imshow('cropped_src_image',cropped_src_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    cropped_src_image = cropped_src_image[::, int(MIN_HOLE_RADIUS_PIXELS)::]  if direction else cropped_src_image[::, 0:int(cropped_src_image.shape[1] - MIN_HOLE_RADIUS_PIXELS)]
    if show: 
        cv2.namedWindow('cropped_src_image', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('cropped_src_image', cropped_src_image.shape[1], cropped_src_image.shape[0])
        cv2.imshow('cropped_src_image',cropped_src_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return cropped_src_image

def filt(src_image, show=False):
    filtered_src_image = src_image.copy()
    nb_iteration = 10
    for i in range(nb_iteration):
        filtered_src_image = cv2.GaussianBlur(filtered_src_image, (5,5), 0) 
    # cv2.imshow('filtered_src_image',filtered_src_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    kernel = np.ones((5, 5), np.uint8)
    filtered_src_image = cv2.morphologyEx(filtered_src_image, cv2.MORPH_CLOSE, kernel, nb_iteration)
    # cv2.imshow("filtered_src_image", filtered_src_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return filtered_src_image



def curve(src_image, show=False):
    processed_src_image = src_image.copy()
    #gray_image = cv2.cvtColor(filtered_src_image, cv2.COLOR_BGR2GRAY)
    mask_image = cv2.inRange(processed_src_image,(0,0,20), (150,255,255))
    mask_rgb_image = cv2.cvtColor(mask_image, cv2.COLOR_GRAY2BGR)
    processed_src_image = processed_src_image & mask_rgb_image
    contours, hierarchy = cv2.findContours(mask_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # get the biggest contour # returns _, contours, _ if using OpenCV 3
    biggest_area = -1
    biggest = None
    for con in contours:
        area = cv2.contourArea(con)
        if biggest_area < area:
            biggest_area = area
            biggest = con
    #print(contours)
    # cv2.imshow('Contour image',frame)
    if biggest is not None and show:
        cv2.drawContours(processed_src_image, [biggest], -1, (0,255,0), 1)
        cv2.imshow('processed_src_image',processed_src_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return biggest


rotated_frame = rotate(unwarped_frame, lowerCenter, upperCenter, False)  

# CAREFUL TAKE BIGGEST OF TWO SIDE
left_cropped_frame = crop(rotated_frame, lowerCenter, upperCenter,  0, True)    
left_filtered_frame = filt(left_cropped_frame, True) 
left_curve_points = curve(left_filtered_frame, True) 

right_cropped_frame = crop(rotated_frame, lowerCenter, upperCenter,  1, True)    
right_filtered_frame = filt(right_cropped_frame, True) 
right_curve_points = curve(right_filtered_frame, True) 



the vectors are= [  0 532] and [ -1 532]
the angle between the two vectors is= 0.10769870683212183
min_y:221
max_y:753
min_x:928
max_x:929
cropped_src_height:532
cropped_src_width:929
min_y:221
max_y:753
min_x:928
max_x:929
cropped_src_height:532
cropped_src_width:976


In [61]:
import numpy as np
import cv2
import glob
from tqdm import tqdm
import pandas as pd
import os
import sys
import shutil
from IPython.display import display


speed_factors = np.array([150, 180, 210, 240, 270, 300, 330, 360])
height_factors = np.array([10, 15, 20, 25])
height_values = np.array([200, 300, 400, 500]) # maybe 198.5?
length_factors = np.array([10, 15, 20, 25, 30, 35])
init_length_pixels = 31
length_pixels = np.array([0, 15, 31, 46, 62, 77]) 
length_pixels += init_length_pixels
length_values = length_pixels * 7
strip_factors = np.array([1, 2, 3, 4])

NB_GROUP = len(speed_factors) * len(strip_factors)
videos_folder = 'frames'

# Define column names
columns = ['height', 'length', 'speed', 'strip', 'group', 'filename', 'lcurve', "rcurve"]

# Create an empty DataFrame with specified column names
df_trials = pd.DataFrame(columns=columns)

for height_index, height_factor in enumerate(height_factors):
    for length_index, length_factor in enumerate(length_factors):
        if length_factor >= height_factor:
            setup_filename = "%dH-%dL" %(height_factor, length_factor)
            setup_path = os.path.join(videos_folder, setup_filename)
            setup_exist = not os.path.isfile(setup_path)
            print("%s -> %s" %(setup_path, setup_exist))
            # Liste des fichiers dans le dossier
            setup_files = os.listdir(setup_path)
            #setup_files = sorted(setup_files)
            # Parcourir les fichiers
            max_setup_group = 0
            for setup_file in setup_files:
                if setup_file.endswith('.jpg') and "edges" not in setup_file:
                    setup_file_path = os.path.join(setup_path, setup_file)
                    # EXTTRACT CURVES
                    frame = cv2.imread(setup_file_path)
                    unwarped_frame = unwarp(frame, False)
                    NB_MARKERS = 2
                    markersCenters, ratioPixelToMillimeters, ratioMillimetersToPixels = loopGetMarkersCenters(unwarped_frame, NB_MARKERS, True)
                    assert NB_MARKERS == len(markersCenters)
                    lowerCenter, upperCenter = getVerticalLine(unwarped_frame,  markersCenters, ratioPixelToMillimeters, show=True)
                    rotated_frame = rotate(unwarped_frame, lowerCenter, upperCenter, True)  
                    left_cropped_frame = crop(rotated_frame, lowerCenter, upperCenter,  0, True)    
                    left_filtered_frame = filt(left_cropped_frame, True) 
                    left_curve_points = curve(left_filtered_frame, True) 

                    right_cropped_frame = crop(rotated_frame, lowerCenter, upperCenter,  1, True)    
                    right_filtered_frame = filt(right_cropped_frame, True) 
                    right_curve_points = curve(right_filtered_frame, True) 


                    #print(setup_file_path)
                    split_setup_file = setup_file.split('_')
                    if len(split_setup_file) >= 3:
                        setup_group = int(split_setup_file[2])
                        max_setup_group = setup_group if setup_group > max_setup_group else max_setup_group

                        new_row = pd.DataFrame({
                            'height':[height_values[height_index]], 
                            'length':[length_values[length_index]],
                            'speed':[speed_factors[int(setup_group/(NB_GROUP-1) * (len(speed_factors) - 1))]],
                            'strip':[setup_group%len(strip_factors)], 
                            'group': [setup_group],
                            'filename': [str(setup_file_path)],
                            'lcurve': [],
                            'rcurve': [],
                        })
                        # Add the new row to the DataFrame
                        df_trials = pd.concat([df_trials, new_row], ignore_index=True)
            if  max_setup_group != 27 and max_setup_group != 31:
                print("nb group is %d" %max_setup_group)
df_trials = df_trials.sort_values(by=['height', 'length', 'speed', 'group', 'strip']) 
print(df_trials)

frames\10H-10L -> True
chessboard found!
chessboard found!
chessboard found!
chessboard found!
chessboard found!
no chessboard found...
chessboard found!
nbMarkersCenters: 2
ratioPixelToMillimeters: 0.984014
ratioMillimetersToPixels: 1.016275
the vectors are= [   0 -532] and [   0 -532]
the angle between the two vectors is= 0.0
cropped_src_height:533
cropped_src_width:933
cropped_src_height:533
cropped_src_width:972


ValueError: All arrays must be of the same length